# Parsing av vedtak

Vi har lastet led alle vedtak og konvert dem til ren tekst. Før vi bruker dem skal vi hente ut noe metadata og unike identifikatoerer. Skriptet under løper igjennom alle vedtaksdokumentene og foretar to operasjoner:

1. Henter ut filnavnet og bruker dette som unik identifikater. Filnavnene er ikke helt uniforme. Noen av de eldre vedtakene har prefikser som 'FLYKN_flykn'; de nyere vedtakene har ikke slike prefisker. Et forsøk på å fjerne disse prefiksene slik at alle id-er skulle komme på et uniformt format, ga negativt resultat: Uten prefiksene var ikke id-ene unike. Vi har derfor beholdt prefiksene slik de forekommer i filnavnene.
2. Henter ut informasjon om hvilken delnemnd vedtaket stammer fra. Denne informasjonen står i overskriften til vedtaket og befinner seg på første eller andre linje i vedtaksteksten. Følgende delnemnder er aktuelle: flyreiser, sjøreiser, kollektivreiser og pakkereiser"

Til slutt lagrer vi all informasjonen i den jsonl-fil, med en linje pr vedtak. I tilegg til hele vedtaksteksten har vi da to felter med medtadata: unik id og nemnd. Dermed er korpuset klart for embedding og lagring i en vektordatabase.

In [29]:
import os
import json

def process_files(directory, keywords, output_file):
    with open(output_file, 'w', encoding='utf-8') as jsonl_file:
        for filename in os.listdir(directory):
            if filename.endswith(".txt"):
                filepath = os.path.join(directory, filename)
                file_id = os.path.splitext(filename)[0]

                try:
                    with open(filepath, 'r', encoding='utf-8') as file:
                        content = file.read()
                        file.seek(0)  # Reset file pointer to start
                        found_nemnd = None
                        lines_to_check = []
                        for line in file:
                            if line.startswith("Vedtak i"):
                                lines_to_check.append(line.strip())
                                # Read the next line if it exists
                                next_line = next(file, '').strip()
                                if next_line:
                                    lines_to_check.append(next_line)
                                # Search for keywords in lines_to_check
                                for keyword in keywords:
                                    if any(keyword in l for l in lines_to_check):
                                        found_nemnd = keyword
                                        break
                                if not found_nemnd:
                                    print(f"No nemnd found for {filename}")
                                break  # Exit after processing 'Vedtak I' and next line
                        else:
                            print(f"'Vedtak i' not found in {filename}")

                    # Write the result to the JSONL file
                    json_obj = {
                        'id': file_id,
                        'nemnd': found_nemnd if found_nemnd else '',
                        'content': content
                    }
                    jsonl_file.write(json.dumps(json_obj, ensure_ascii=False) + '\n')

                except Exception as e:
                    print(f"Error processing file {filename}: {e}")

# Set the directory and keywords
directory_path = './transportklagenmnda_txt/'
nemnd_keywords = ["Fly", "Sjø", "Kollektivreiser", "Pakkereisenemnda"]

# Process the files and save to 'vedtak.jsonl' in the current directory
process_files(directory_path, nemnd_keywords, 'vedtak.jsonl')
